In [46]:
from __future__ import print_function 

%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
%matplotlib inline

import os, sys
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras import backend as K

import pandas as pd
import pickle
import gc, re, copy
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils

# Project imports 
from data import mnist_m as mnistm
from data import mnist
from data.label_shift import label_shift_linear, plot_labeldist, plot_splitbars
from data.tasks import load_task
from experiments.training import *
from experiments.SL_bound import *
from experiments.DA_bound import *
from bounds.bounds import *
from util.kl import *
from util.misc import *
from util.batch import *
from results.plotting import *

# Hyper-parameters
task = 2
seed = 69105
n_classifiers = 20
delta=0.05 ## what would this be?   
binary=True
bound='germain'
epsilons=[0.01]
alphas=[0.1]#0,0.3]
sigmas=[[3,2],[1,2],[3,3]]

project_folder = "/cephyr/users/frejohk/Alvis/projects/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Alvis params

In [47]:
project = 'SNIC2021-7-82'
username = 'frejohk'
job = 'batch_bound_single.sbatch'

In [48]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            103465     alvis     bash  frejohk  R    6:25:56      1 alvis1-08 


In [49]:
!cat $job

#!/usr/bin/bash

#SBATCH -t 1-00:00:00
#SBATCH -N 1 --gpus-per-node=T4:1
#SBATCH -p alvis 

if [ -z "$task" ]
then
    task=2
fi
if [ -z "$seed" ]
then 
    seed=69105
fi
if [ -z "$alpha" ]
then 
    alpha=0.0
fi
if [ -z "$sigma" ]
then 
    sigma='3,3'
fi
if [ -z "$epsilon" ]
then 
    epsilon=0.01
fi
if [ -z "$delta" ]
then 
    delta=0.05
fi
if [ -z "$binary" ]
then 
    binary=0
fi
if [ -z "$nclassifiers" ]
then 
    nclassifiers=2
fi
if [ -z "$bound" ]
then 
    bound='germain'
fi
if [ -z "$prior" ]
then 
    prior=''
fi
if [ -z "$posterior" ]
then 
    posterior=''
fi

. load_modules.sh

python batch_bound_single.py -t $task -r $seed -a $alpha -s $sigma -e $epsilon -d $delta -b $binary -n $nclassifiers -B $bound -p $prior -P $posterior

In [50]:
print('Iterating over experiments...\n')

fids = []

os.makedirs('logs', exist_ok=True)


np.random.seed(seed)
for alpha in alphas:
    
    print("alpha:"+str(alpha))
    
    for epsilon in epsilons:
        print("  epsilon:"+str(epsilon))
        for sigma in sigmas:    
            print("    sigma:"+str(sigma))
            arg_list = get_job_args(task, bound=bound, alpha=alpha, sigma=sigma,
                                    epsilon=epsilon, binary=binary, n_classifiers=n_classifiers)
            for a in arg_list:   
                
                ckpt = os.path.splitext(os.path.basename(a['posterior_path']))[0]
                fid = './logs/batch_t-%d_r-%d_a-%.4f_s-%d%d_e-%.4f_d-%.4f_b-%d_n-%d_B-%s_c-%s' % \
                    (task, seed, alpha, sigma[0], sigma[1], epsilon, delta, binary, n_classifiers, bound, ckpt)
                
                sigstr = '"%d.%d"' % (sigma[0], sigma[1])
                exp = 'task=%d,seed=%d,alpha=%.4f,sigma="%s",epsilon=%.4f,delta=%.4f,'% (task, seed, alpha, sigstr, epsilon, delta)\
                        +'binary=%d,nclassifiers=%d,bound=%s,prior=%s,posterior=%s' % (binary, n_classifiers, bound, a['prior_path'], a['posterior_path'])
                
                prior_path = a['prior_path']
                output = !sbatch -o "$fid"-%j.out -e "$fid"-%j.err -A $project --export="$exp" $job

                jobid = int(output[0].split(' ')[-1])
                fid = fid+'-%s' % jobid

                fids.append(fid)

Iterating over experiments...

alpha:0.1
  epsilon:0.01
    sigma:[3, 2]
    sigma:[1, 2]
    sigma:[3, 3]


In [51]:
print(fids)

['./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_0-103959', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_45-103960', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_90-103961', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_135-103962', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_180-103963', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_225-103964', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_270-103965', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_315-103966', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_360-103967', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_405-103968', './logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n

In [64]:
!squeue -u $username

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
            104039     alvis batch_bo  frejohk PD       0:00      1 (Priority) 
            104038     alvis batch_bo  frejohk PD       0:00      1 (Priority) 
            104036     alvis batch_bo  frejohk  R       5:02      1 alvis2-08 
            104037     alvis batch_bo  frejohk  R       5:02      1 alvis2-08 
            104015     alvis batch_bo  frejohk  R       5:32      1 alvis2-03 
            104016     alvis batch_bo  frejohk  R       5:32      1 alvis2-08 
            104017     alvis batch_bo  frejohk  R       5:32      1 alvis2-12 
            104018     alvis batch_bo  frejohk  R       5:32      1 alvis2-12 
            104019     alvis batch_bo  frejohk  R       5:32      1 alvis2-14 
            104020     alvis batch_bo  frejohk  R       5:32      1 alvis2-15 
            104021     alvis batch_bo  frejohk  R       5:32      1 alvis2-15 
            104022     alvis batch_bo  frej

In [62]:
for fid in fids: 
    print('------')
    print(fid)
    !tail -n 50 "$fid".out
    !tail -n 5 "$fid".err
    print(' \n\n')

------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_0-103959
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103959&from=1632841914000
{'task': 2, 'seed': 69105, 'alpha': 0.1, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 20, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/10/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_10/1_0.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883]]
M

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_180-103963
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103963&from=1632841913000
{'task': 2, 'seed': 69105, 'alpha': 0.1, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 20, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/10/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_10/1_180.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.002

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_360-103967
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103967&from=1632841916000
{'task': 2, 'seed': 69105, 'alpha': 0.1, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 20, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/10/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_10/1_360.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.002

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_540-103971
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103971&from=1632841914000
{'task': 2, 'seed': 69105, 'alpha': 0.1, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 20, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/10/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_10/1_540.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.002

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-32_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_4-103975
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=103975&from=1632841914000
{'task': 2, 'seed': 69105, 'alpha': 0.1, 'sigma': '3.2', 'epsilon': 0.01, 'delta': 0.05, 'binary': 1, 'n_classifiers': 20, 'bound': 'germain', 'prior_path': 'priors/task2/Binary/10/prior.ckpt', 'posterior_path': 'posteriors/task2/Binary/10_10/2_4.ckpt'}
Loading data...
mean, variance 0.36348352 70.18035
---------------Load MNIST----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)


mean, variance 1.1809415 74.36859
---------------Load MNIST-M----------------
Training set (60000, 32, 32, 3) (60000, 10)
Test set (10000, 32, 32, 3) (10000, 10)
[[10.986111, 5.0, 2.999428, 1.99958, 1.399789, 1.0, 0.71435696, 0.5, 0.33346358, 0.20003448], [0.09088036, 0.19987813, 0.33326975, 0.5, 0.7143216, 1.0, 1.4001397, 2.0, 3.0, 5.0025883

...

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-12_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_12-104010
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104010&from=1632842606000
2021-09-28 17:24:40.255982: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14158 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0000:d8:00.0, compute capability: 7.5)
2021-09-28 17:24:40.256366: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-09-28 17:24:40.546495: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open priors/task2/Binary/10/prior.ckpt: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
2021-09-28 17:24:41.454021:

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_180-104017
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104017&from=1632842635000
2021-09-28 17:25:10.139030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14158 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0000:07:00.0, compute capability: 7.5)
2021-09-28 17:25:10.139413: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-09-28 17:25:10.399873: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open priors/task2/Binary/10/prior.ckpt: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
2021-09-28 17:25:11.232355

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-1_495-104024
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104024&from=1632842636000
2021-09-28 17:25:09.942263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14158 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0000:87:00.0, compute capability: 7.5)
2021-09-28 17:25:09.942620: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-09-28 17:25:10.205768: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open priors/task2/Binary/10/prior.ckpt: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
2021-09-28 17:25:11.132881

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_6-104031
This job can be monitored from: https://scruffy.c3se.chalmers.se/d/alvis-job/alvis-job?var-jobid=104031&from=1632842638000
2021-09-28 17:25:11.913485: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1241] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14158 MB memory) -> physical GPU (device: 0, name: Tesla T4, pci bus id: 0000:87:00.0, compute capability: 7.5)
2021-09-28 17:25:11.913773: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-09-28 17:25:12.184470: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open priors/task2/Binary/10/prior.ckpt: Data loss: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?
2021-09-28 17:25:13.200412: 

 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_13-104038
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_13-104038.out' for reading: No such file or directory
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_13-104038.err' for reading: No such file or directory
 


------
./logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_14-104039
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_14-104039.out' for reading: No such file or directory
tail: cannot open './logs/batch_t-2_r-69105_a-0.1000_s-33_e-0.0100_d-0.0500_b-1_n-20_B-germain_c-2_14-104039.err' for reading: No such file or directory
 


